In [1]:
%load_ext autoreload
import pandas as pd
import numpy as np

# Loading the Data

In [2]:
X_train_1 = pd.read_csv('data/dengue_features_train.csv')
y_train = pd.read_csv('data/dengue_labels_train.csv')['total_cases']
attr = list(X_train_1)
attr

['city',
 'year',
 'weekofyear',
 'week_start_date',
 'ndvi_ne',
 'ndvi_nw',
 'ndvi_se',
 'ndvi_sw',
 'precipitation_amt_mm',
 'reanalysis_air_temp_k',
 'reanalysis_avg_temp_k',
 'reanalysis_dew_point_temp_k',
 'reanalysis_max_air_temp_k',
 'reanalysis_min_air_temp_k',
 'reanalysis_precip_amt_kg_per_m2',
 'reanalysis_relative_humidity_percent',
 'reanalysis_sat_precip_amt_mm',
 'reanalysis_specific_humidity_g_per_kg',
 'reanalysis_tdtr_k',
 'station_avg_temp_c',
 'station_diur_temp_rng_c',
 'station_max_temp_c',
 'station_min_temp_c',
 'station_precip_mm']

## Cleaning the noisy training data

In [3]:
def bools_to_indexes(booleans):
    r = []
    for idx, x in enumerate(booleans):
        if x:
            r.append(idx)
    return r

idx = bools_to_indexes(X_train_1['weekofyear'] == 53)
y_train.drop(idx, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_train_1.drop(idx, inplace=True)
X_train_1.reset_index(drop=True, inplace=True)
X_train_1.shape

(1451, 24)

# Model Selection

In [3]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
score_metric='neg_mean_absolute_error'
jobs=-1 #-1 to make it execute in parallel
verbose_level = 0
random_n = 42
base_args = {'estimator': None, 'param_distributions': None, 'n_iter': None, 'scoring': score_metric, 'n_jobs': jobs, 'cv': None, 'verbose': verbose_level, 'random_state': random_n, 'return_train_score': True, 'iid': True}

## SVR
* The results with the kernel *sigmoid* and *poly* were too bad, so we removed them.

In [4]:
k_folds=10
n_iter_search = 20
C = sp_randint(0, 10000)
params = {'kernel':['linear'], 'gamma':['scale'], 'C': C}
SVR_optimizer = RandomizedSearchCV(estimator=SVR(), param_distributions=params, n_iter=n_iter_search, scoring=score_metric, n_jobs=jobs, cv=k_folds, verbose=verbose_level, random_state=random_n, return_train_score=True, iid=True)

## Regression Trees
* 18.01 - with 2 previous weeks & without PCA & with (max_depth=6, min_samples_leaf=0.1611807565247405, min_samples_split=0.11193019906931466)
* 18.29 - With PCA at 0.9
* 18.27 - With PCA at 0.95
* 18.36 - With PCA at 0.65. PCA appears to be only making the model worse.
* 18.38 - Without PCA and with previous weeks. Clearly the previous weeks are useful
* 17.87 - Without PCA and with 3 previous weeks
* 17.86 - Without PCA and with 4 previous weeks
* 18.28 - With PCA 0.95 and 3 previous weeks fixed
* 9.16 - Without PCA, with 3 weeks and 1 last infection (max_depth=5, min_samples_leaf=0.03, min_samples_split=0.108)
* **9.04** - Without PCA, with 3 weeks and 1 last infection (max_depth=5, min_samples_leaf=0.03, min_samples_split=0.108)

In [5]:
k_folds=10
n_iter_search = 100
min_samples = sp_uniform(0.01, 0.35)
params = {'criterion':['mae'], 'max_depth': sp_randint(2, 10), 'min_samples_split': min_samples, 'min_samples_leaf': min_samples}
Tree_optimizer = RandomizedSearchCV(estimator=DecisionTreeRegressor(), param_distributions=params, n_iter=n_iter_search, scoring=score_metric, n_jobs=jobs, cv=k_folds, verbose=verbose_level, random_state=random_n, return_train_score=True, iid=True)

## Random Forests
* 18.34 With 4 previous weeks and without PCA
* 17.79 With fixed 3 previous weeks and PCA at 0.95 (n_estimators= ?, max_depth = 2, min_samples_leaf=0.112, min_samples_split=0.224)
* 17.74 With fixed 3 previous weeks and without PCA (n_estimators= 13 max_depth = 5, min_samples_leaf=0.09, min_samples_split=0.24)
* **9.13** with 3 previous weeks and 1 last infected (n_estimators=9 max_depth = 9, min_samples_leaf=0.014, min_samples_split=0.07)
* 9.22 with 3 previous weeks and 3 last infected (n_estimators=9 max_depth = 9, min_samples_leaf=0.014, min_samples_split=0.08)

In [6]:
k_folds=10
n_iter_search = 40
params = {'n_estimators': sp_randint(2,50), 'criterion':['mae'], 'max_depth': sp_randint(2, 10)}
Forest_optimizer = RandomizedSearchCV(estimator=RandomForestRegressor(n_jobs=-1), param_distributions=params, n_iter=n_iter_search, scoring=score_metric, n_jobs=jobs, cv=k_folds, verbose=verbose_level, random_state=random_n, return_train_score=True, iid=True)

## Adaboost of Trees
* 10.78 - With 3 last weeks a 3 last infected 
* **8.49** - With 3 last weeks a 3 last infected and only max_depth tuned.

In [7]:
k_folds=10
n_iter_search = 20
params = {'n_estimators': sp_randint(40, 100), 'base_estimator__criterion':['mae'], 'base_estimator__max_depth': sp_randint(2,7)}
AdaTree_optimizer = RandomizedSearchCV(estimator=AdaBoostRegressor(base_estimator=DecisionTreeRegressor()), param_distributions=params, n_iter=n_iter_search, scoring=score_metric, n_jobs=jobs, cv=k_folds, verbose=verbose_level, random_state=random_n, return_train_score=True, iid=True)

## KNN
* 21.349 - with PCA at 0.65 & 2 previous weeks
* 20.36  - without PCA

In [8]:
k_folds=10
n_iter_search = 100
params = {'n_neighbors': sp_randint(3,150), 'weights': ['uniform', 'distance']}
KNN_optimizer = RandomizedSearchCV(estimator=KNeighborsRegressor(n_jobs=-1), param_distributions=params, n_iter=n_iter_search, scoring=score_metric, n_jobs=jobs, cv=k_folds, verbose=verbose_level, random_state=random_n, return_train_score=True, iid=True)

# Optimization
* Interestingly, PCA makes all the models worst in this case.
* After the exaustive search, the best model was the SVR which obtained an MAE of 6.52.

In [15]:
%autoreload
from OurPipeline import create_pipeline
from sklearn.decomposition import PCA

optimizers=[Tree_optimizer, Forest_optimizer, AdaTree_optimizer, KNN_optimizer]#, SVR_optimizer]
weeks = [1]
weeks_infected = [3]
pca = [None]

n_total = len(optimizers) * len(weeks) * len(weeks_infected) * len(pca)

results=[]
best_attempt = None
best_score = np.inf
idx=0
for opt in optimizers:
    for w in weeks:
        for wi in weeks_infected:
            for p in pca:
                pipeline = create_pipeline(attr, n_weeks=w, n_weeks_infected=wi, estimator_optimizer=opt, add_noise=True, noise_mean=6.5, noise_std=6.5, pca=None)
                pipeline.fit(X_train_1, y_train)
                score = pipeline.named_steps['est_opt'].best_score_
                best_estimator = pipeline.named_steps['est_opt'].best_estimator_
                attempt = [best_estimator, w, wi, p, score]
                if abs(score) < best_score:
                    best_score = abs(score)
                    best_attempt = attempt
                    print('\nBest score of {} with the estimator {}'.format(best_score, best_estimator))
                idx+=1
                print(str(idx) + '/' + str(n_total), end='\t')
                results.append(attempt)


Best score of 10.757898351648352 with the estimator DecisionTreeRegressor(criterion='mae', max_depth=7, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=0.048518173584686866,
           min_samples_split=0.08977730688967958,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')
1/4	
Best score of 8.57545699492815 with the estimator RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=26, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
2/4	3/4	4/4	

In [ ]:
pd.DataFrame(results, columns=['estimator', 'weeks', 'weeks_infected', 'PCA', 'score'])

In [37]:
best_attempt

[SVR(C=5191, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
   kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
 1,
 3,
 None,
 -6.522347109745663]

# Predict

In [171]:
%autoreload
from OurPipeline import create_pipeline

pipeline = create_pipeline(attr, n_weeks=1, n_weeks_infected=3, pca=None)
X_train = pipeline.fit_transform(X_train_1, y_train)

In [56]:
model = SVR(kernel= 'linear', C=5191, gamma='scale')
model.fit(X_train, y_train)

SVR(C=5191, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

### Loading test data

In [10]:
X_test_1 = pd.read_csv('data/dengue_features_test.csv')
print(X_test_1.shape)

(416, 24)


## One by one prediction
* Given that we are making sequential predictions, i.e.: the prediction from a week relies on the prediction from the previous weeks, we must make the transformations and predictions one by one.
* Given that this kind of prediction is very prone to a snowball effect on errors our first solution had an error of 26. To solve this we came up with the idea of adding noise to the train data. However for this solution we need to know both: the mean of the error and its standard deviation (*std*). We already know the mean (MAE), we just need to know the *std*

In [172]:
predictions=[]
for idx in range(X_test_1.shape[0]):
    x = pipeline.transform(X_test_1.loc[idx:idx,:])
    pred = model.predict(x)
    pred = int(np.round(pred))
    pipeline.named_steps['l_infected'].append_y(pred)
    predictions.append(pred)
len(predictions)

416

### Calculating an approximation of the *std*
* It is approximately 10.9. We can see that the MAE is close to the one calculated in the cross-validation.

In [216]:
%autoreload
from OurPipeline import create_pipeline
from sklearn.model_selection import ShuffleSplit

pipeline = create_pipeline(attr, n_weeks=1, n_weeks_infected=3, pca=None)
X_train = pipeline.fit_transform(X_train_1, y_train)

In [217]:
sp = ShuffleSplit(n_splits=1, train_size=1000, test_size=None, random_state=random_n)
for train, test in sp.split(X_train, y_train):
    X_train_std = X_train[train]
    y_train_std = y_train[train]
    X_test_std = X_train[test]
    y_test_std = y_train[test]
X_train_std.shape, y_train_std.shape
X_test_std.shape, y_test_std.shape

In [222]:
model = SVR(kernel= 'linear', C=5191, gamma='scale')
model.fit(X_train_std, y_train_std)

SVR(C=5191, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [234]:
predictions = model.predict(X_test_std)
predictions = list(map(lambda x: int(np.round(x)), predictions))
errors = list(map(abs, predictions - y_test_std))
np.mean(errors), np.std(errors)

(6.7785087719298245, 10.959317651673116)

# One by one prediction with noise

In [11]:
%autoreload
from OurPipeline import create_pipeline

pipeline = create_pipeline(attr, n_weeks=1, n_weeks_infected=3, add_noise=True, noise_mean=6.5, noise_std=6.5, pca=None)
X_train = pipeline.fit_transform(X_train_1, y_train)

In [12]:
SVR_optimizer.fit(X_train, y_train)
model=SVR_optimizer.best_estimator_
SVR_optimizer.best_score_

-8.436222184081323

In [13]:
model.fit(X_train, y_train)

SVR(C=1685, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [14]:
predictions=[]
for idx in range(X_test_1.shape[0]):
    x = pipeline.transform(X_test_1.loc[idx:idx,:])
    pred = model.predict(x)
    pred = int(np.round(pred))
    pipeline.named_steps['l_infected'].append_y(pred)
    predictions.append(pred)
len(predictions)

416

## Submission

In [17]:
submit = pd.DataFrame(predictions, columns=['total_cases'])
x_3 = X_test_1.iloc[:,:3].copy()
submit = pd.concat([x_3, submit], axis=1)
submit.to_csv('data/submit.csv', index=False)